# Statistics

**Instalation of the packages**

In [ ]:
pip install "statsmodels"

## Read and merge files

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import re
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
Sub1 = pd.read_csv("path...\KubiosHRVresultsSub1.csv", header = 1) #read data file
Sub2 = pd.read_csv("path...\KubiosHRVresultsSub2.csv", header = 1) #read data file
Sub3 = pd.read_csv("path...\KubiosHRVresultsSub3.csv", header = 1) #read data file
#... add all files from Kubios HRV

In [ ]:
df = pd.concat([Sub1,Sub2,Sub3], axis = 0) # merge all files

In [ ]:
df # visualize

In [ ]:
cols = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
df = df.drop(df.columns[cols],axis=1) # eliminate informational columns

In [ ]:
df.drop(columns=["Unnamed: 103"], inplace=True) # eliminate "random" column

In [ ]:
df = df.set_index("FileName") #set the ne indices

In [ ]:
df_Emp = df.filter(like = "Empatica", axis = 0)
df_Far = df.filter(like = "Faros", axis = 0) #divide our datasets depending on the source

## Statistical Tests

In [ ]:
df_Emp #visualize

In [ ]:
df_Far #visualize

### Mean Difference - Global

In [ ]:
intro = pd.Index(df_Emp.columns) #create index for iteration process

In [ ]:
data = []
data1 = []
data2 = [] #create empty lists to save the data
for column in df_Emp.columns: #mean difference between the complete datasets
        x = df_Emp[column].astype(float).mean() #list the means of Empatica
        y = df_Far[column].astype(float).mean() #list the means of Faros
        z = (df_Emp[column].astype(float).mean() - df_Far[column].astype(float).mean()) #list the difference of the means
        z = abs (z)
        data.append (x)
        data1.append (y)
        data2.append (z)
mean_x = pd.DataFrame (data, index = intro, columns=["Means Empatica"])
mean_y = pd.DataFrame (data1, index = intro, columns=["Means Faros"])
mean_z = pd.DataFrame (data2, index = intro, columns=["Mean Difference"])
#generate df's
mean_diff = pd.concat ([mean_x,mean_y,mean_z], axis = 1) #concat our 3 datasets
mean_diff.index.name = "Name" #give name to the index
mean_diff.rename(index=lambda x: x.replace('(',''), inplace=True)
mean_diff.rename(index=lambda x: x.replace(')',''), inplace=True) #remove the "(" and ")" 's
mean_diff #visualize

## Conservative approach with NaN

Check number of NaN's

In [ ]:
df_Emp.isnull().sum().sum() #count NaN's in Empatica

In [ ]:
df_Far.isnull().sum().sum() #count NaN's in Faros

Here the idea is to maintain as much data as possible, for this reason we not delete the whole column with NaN values, rather than only
the specific value and its counterpart in the other DF (because is not longer possible to do a comparison)

In [ ]:
df1 = df_Emp.astype(float)
df2 = df_Far.astype(float) # transform all our values to floats

In [ ]:
df1.rename(columns=lambda x: x.replace('(',''), inplace=True)
df1.rename(columns=lambda x: x.replace(')',''), inplace=True) 
df2.rename(columns=lambda x: x.replace('(',''), inplace=True)
df2.rename(columns=lambda x: x.replace(')',''), inplace=True) 

## Alter the label of our datasets

In [ ]:
df1.rename(index=lambda x: x.replace('Sub1_Empatica_','Sub1_'), inplace=True)
df1.rename(index=lambda x: x.replace('Sub2_Empatica_','Sub2_'), inplace=True)
df1.rename(index=lambda x: x.replace('Sub3_Empatica_','Sub3_'), inplace=True)

df2.rename(index=lambda x: x.replace('Sub1_Faros_','Sub1_'), inplace=True )
df2.rename(index=lambda x: x.replace('Sub2_Faros_','Sub2_'), inplace=True )
df2.rename(index=lambda x: x.replace('Sub3_Faros_','Sub3_'), inplace=True )
# edit the beginning of the index of our df's, depends of how was save in the files

In [ ]:
df12_merged = df1.merge(df2, suffixes=[" _empatica", " _faros"], on="FileName") # merge the datasets

In [ ]:
df12_merged

Fix the names of our columns (remove "(" and ")")

In [ ]:
df12_merged.rename(columns=lambda x: x.replace('(',''), inplace=True)
df12_merged.rename(columns=lambda x: x.replace(')',''), inplace=True) 

Create our working list for itemize in the loop

In [ ]:
w = df_Emp #copy one of the original datasets

In [ ]:
w.rename(columns=lambda x: x.replace('(',''), inplace=True)
w.rename(columns=lambda x: x.replace(')',''), inplace=True) #remove the "(" and ")" 's to avoid RE expressions

In [ ]:
column_names = w.columns # create a list for reference

## Data exploration

We want to explore the distribution of our dataset

In [ ]:
skew_emp = pd.DataFrame ((df1.loc[ : , df1.columns ].skew(axis = 0, skipna = True)), columns=["Skew_Emp"])
skew_emp.index.name = "HRV metric" #give name to the index
skew_far = pd.DataFrame ((df2.loc[ : , df2.columns ].skew(axis = 0, skipna = True)), columns = ["Skew_Far"])
skew_far.index.name = "HRV metric" #give name to the index
skew_tot = skew_emp.merge(skew_far, on = "HRV metric")
skew_tot #skewness of the data

If the skewness is between -0.5 and 0.5, the data are fairly symmetrical
If the skewness is between -1 and – 0.5 or between 0.5 and 1, the data are moderately skewed
If the skewness is less than -1 or greater than 1, the data are highly skewed

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_Emp"]) > 0.5] # number of values moderately skewed for Empatica

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_Emp"]) > 1] # number of values highly skewed for Empatica

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_Far"]) > 0.5] # number of values moderately skewed for Empatica

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_Far"]) > 1] # number of values highly skewed for Empatica

### Evaluate the normallity of our dataset

We can also explore the distribution with Shapiro-Wilk test

In [ ]:
shap1 = []
s_val_p1 = []
colnew_s1 =[] #create empty lists to save the data
for column in column_names: #iterate in our reference list
        x = df1.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
            r, p = stats.shapiro(x) #shapiro test
            shap1.append (r)
            s_val_p1.append (p)
            colnew_s1.append (column) # add the specified values to the list
res_shap1 = pd.DataFrame (shap1, columns=["Shap_statistics_emp"])
res_shap2 = pd.DataFrame (s_val_p1, columns=["P-value(Shap_stats_emp)"])
res_shap3 = pd.DataFrame (colnew_s1, columns=["Name"]) # create df from lists
shap_stats1 = pd.concat([res_shap1,res_shap2,res_shap3], axis =1) #merge into one


shap2 = []
s_val_p2 = []
colnew_s2 =[] #create empty lists to save the data
for column in column_names: #iterate in our reference list
        x = df2.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
            r, p = stats.shapiro(x) #shapiro test
            shap2.append (r)
            s_val_p2.append (p)
            colnew_s2.append (column) # add the specified values to the list
res_shap4 = pd.DataFrame (shap2, columns=["Shap_statistics_far"])
res_shap5 = pd.DataFrame (s_val_p2, columns=["P-value(Shap_stats_far)"])
res_shap6 = pd.DataFrame (colnew_s2, columns=["Name"]) # create df from lists
shap_stats2 = pd.concat([res_shap4,res_shap5,res_shap6], axis =1) #merge into one


shap_stats_tot = pd.merge(shap_stats1, shap_stats2, how= "outer", on="Name")  #merge both files
shap_stats_tot.set_index("Name", inplace = True)
shap_stats_tot


### Pearson correlation

In [ ]:
corr = []
p_val_p = []
colnew_p =[] #create empty lists to save the data
for column in column_names: #iterate in our reference list
        x = df12_merged.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
                r, p = stats.pearsonr(x[x.columns[0]], x[x.columns[1]]) #Pearson correlation
                corr.append (r)
                p_val_p.append (p)
                colnew_p.append (column) # add the specified values to the list
res_per1 = pd.DataFrame (corr, columns=["Pearson_correlation_value"])
res_per2 = pd.DataFrame (p_val_p, columns=["P-value(Pear-corr)"])
res_per3 = pd.DataFrame (colnew_p, columns=["Name"]) # create df from lists
per_corr = pd.concat([res_per1,res_per2,res_per3], axis =1) #merge into one
per_corr

### Student t-test

In [ ]:
stut = []
p_val_s = []
colnew_s =[]#create empty lists to save the data
for column in column_names: #iterate in our reference list
        x = df12_merged.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
                r, p = stats.ttest_rel(x[x.columns[0]], x[x.columns[1]]) #paired test
                stut.append (r)
                p_val_s.append (p)
                colnew_s.append (column)# add the specified values to the list
res_stut1 = pd.DataFrame (stut, columns=["T-test_value"])
res_stut2 = pd.DataFrame (p_val_s, columns=["P-value(T-Test)"])
res_stut3 = pd.DataFrame (colnew_s, columns=["Name"])# create df from lists
t_test = pd.concat([res_stut1,res_stut2,res_stut3], axis =1) #merge into one
t_test

### Wilcoxon Rank test

In [ ]:
wilc = []
p_val_w = []
colnew_w =[]#create empty lists to save the data
for column in column_names: #iterate in our reference list
        x = df12_merged.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
            r, p = stats.wilcoxon(x[x.columns[0]], x[x.columns[1]], alternative='two-sided') #wilcoxon test
            wilc.append (r)
            p_val_w.append (p)
            colnew_w.append (column)# add the specified values to the list
res_wilc1 = pd.DataFrame (wilc, columns=["Wilconxon_stats"])
res_wilc2 = pd.DataFrame (p_val_w, columns=["P-value(Wilcoxon)"])
res_wilc3 = pd.DataFrame (colnew_w, columns=["Name"])# create df from lists
wilc_rank = pd.concat([res_wilc1,res_wilc2,res_wilc3], axis =1) #merge into one
wilc_rank

## Create a comparison complete table

In [ ]:
per_corr.set_index("Name", inplace=True)
t_test.set_index("Name", inplace=True) # set the index

In [ ]:
per_corr #visualize

In [ ]:
t_test #visualize

In [ ]:
mean_diff #visualize

In [ ]:
per_t_compared = pd.merge(per_corr, t_test, how= "outer", on="Name") 
#merge pearson df's using the combine column on name

In [ ]:
per_t_compared #visualize

In [ ]:
non_wilc_compared = pd.merge(per_t_compared, mean_diff, how= "outer", on="Name") 
#merge pearson df's using the combine column on name

In [ ]:
total_compared = pd.merge(non_wilc_compared, wilc_rank, how= "outer", on="Name") 
#merge pearson df's using the combine column on name

In [ ]:
total_compared #visualize

In [ ]:
total_compared.to_csv("path...comparison_table.csv", decimal=',', sep=';', index = False)
#save

### Bland-Altman Plot

In [ ]:
font1 = {'family':'serif','color':'blue','size':20}

for column in column_names:
    d = df12_merged.filter(like=column)  #filter from the combine dataset only the columns that start with the "name" at a time
    d = d.dropna() #drop the NaN for the specified duplet
    if len(d) > 2:    # setting a threshold of at least 3 values
        #create Bland-Altman plot                  
        f, ax = plt.subplots(1, figsize = (8,5))
        sm.graphics.mean_diff_plot(d[d.columns[0]], d[d.columns[1]], ax = ax)        
        plt.title(column, fontdict = font1)
        #display Bland-Altman plot
        plt.show()